In [ ]:
import openai


In [1]:
with open("raw_Metadata_Image8_full.txt") as f:
    metadata = f.read()

In [3]:
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please transcribe the following metadata such that it follows the latest ome xml schema."},
    {"role": "assistant", "content": "Sure, please provide the raw metadata such that I can transcribe it."},
    {"role": "user", "content": metadata}
]

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=messages,
    temperature=0,
    max_tokens=5000,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\"\"\""]
)
print(response["choices"][0]["message"]["content"])

Here is the transcribed metadata following the latest OME XML schema:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
  <Image ID="Image_8">
    <Pixels BigEndian="false" DimensionOrder="XYCZT" Interleaved="false" PixelType="uint8" SizeC="3" SizeT="30" SizeX="680" SizeY="280" SizeZ="1">
      <Channel ID="Channel_1" SamplesPerPixel="1">
        <LightPath>
          <IlluminationChannel ID="IlluminationChannel_1">
            <LightSourceRef ID="LightSource_1"/>
            <DetectorRef ID="Detector_1"/>
          </IlluminationChannel>
        </LightPath>
      </Channel>
      <Channel ID="Channel_2" SamplesPerPixel="1">
        <LightPath>
          <IlluminationChannel ID="IlluminationChannel_2">
            <LightSourceRef ID="LightSource

In [4]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

client = instructor.patch(OpenAI())

class UserDetail(BaseModel):
    name: str
    age: int
    
user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Aaron is 25 years old"},
    ],
)
assert isinstance(user, UserDetail)
assert user.name == "Aaron"
assert user.age == 25
print(user.model_dump_json(indent=2))

{
  "name": "Aaron",
  "age": 25
}


In [5]:
print(user._raw_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-9HU1jwSUryakJkn3BflKfRzhwhfBZ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_w4tRlHdD10JWUqfrBnLGasxT",
            "function": {
              "arguments": "{\"name\":\"Aaron\",\"age\":25}",
              "name": "UserDetail"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1713954027,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_c2295e73ad",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 81,
    "total_tokens": 90
  }
}


In [18]:
from openai import OpenAI
from pydantic import BaseModel
import instructor

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())

class QuestionAnswer(BaseModel):
    question: str
    answer: str

question = "What is the meaning of life?"
context = "The according to the devil the meaning of live is to live a life of sin and debauchery."

qa: QuestionAnswer = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=QuestionAnswer,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
        },
        {
            "role": "user",
            "content": f"using the context: {context}\n\nAnswer the following question: {question}",
        },
    ],
)
print(qa.model_dump_json(indent=2))

{
  "question": "What is the meaning of life?",
  "answer": "According to the devil, the meaning of life is to live a life of sin and debauchery."
}


# LLM_Validator validation doesnt seem to work so well for me :(

In [20]:
from pydantic import BaseModel, BeforeValidator
from typing_extensions import Annotated
from instructor import llm_validator

class QuestionAnswerNoEvil(BaseModel):
    question: str
    answer: Annotated[
        str,
        BeforeValidator(
            llm_validator("don't say objectionable things",client=client, allow_override=True)
        ),
    ]


try:
    qa: QuestionAnswerNoEvil = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=QuestionAnswerNoEvil,
        messages=[
            {
                "role": "system",
                "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
            },
            {
                "role": "user",
                "content": f"using the context: {context}\n\nAnswer the following question: {question}",
            },
        ],
    )
except Exception as e:
    print(e)
    
print(qa.model_dump_json(indent=2))

{
  "question": "What is the meaning of life?",
  "answer": "The meaning of life, according to the devil, is to live a life of sin and debauchery."
}


In [53]:
import datetime
from openai import OpenAI
from typing import List
from pydantic import Field
from typing import Iterable, Optional

client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)


In [74]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str

class Person(BaseModel):
    name: str
    age: int
    birthday: datetime.date
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 2"
    )
    
class MaybePerson(BaseModel):
    result: Optional[Person] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]
    
    
schema = OME.model_json_schema()


def extract(content):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user",
             "content": f"Extract '{content}'"}],
        response_model=MaybePerson
    )

In [77]:
content = f"Aaron is 25 years old his birthday was yesterday today is {datetime.datetime}. He is friends with Caro whichs brithday was last week. They both like json."
extract(content).model_dump()

{'result': {'name': 'Aaron',
  'age': 25,
  'birthday': datetime.date(1997, 5, 12),
  'properties': [{'index': '1', 'key': 'friends_with', 'value': 'Caro'}]},
 'error': False,
 'message': 'Extracted person information successfully.'}